In [75]:
# importuje niezbędne biblioteki
import re
from datetime import datetime
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import os, sys
import glob
from matplotlib.backends.backend_pdf import PdfPages
import time

In [76]:
# Tworzymy listę i dwa puste DF
listOfFiles = []
DFL = pd.DataFrame()
DFM = pd.DataFrame() 

In [77]:
# Funkcja pobiera listę ścieżek
def get_filenames():
    Tk().withdraw()
    print("Initializing Dialogue... \nPlease select a file.")
    tk_filenames = filedialog.askdirectory()
    tempDir = tk_filenames
    return tempDir


choosen_dir = get_filenames()
# glob zbiera listę plików 
os.chdir(choosen_dir)
for file in glob.glob("*.csv"):
    listOfFiles.append(file)

# Do wcześniej stworzonych DF dodajemy kolejno pliki w zależności od tego na jaką literę plik się zaczyna.
for file in listOfFiles:
    if file.startswith('L'):
        Table = pd.read_csv(file, sep=',')
        DF1 = pd.DataFrame(Table)
        DFL = DFL.append(DF1, sort=False)
    elif file.startswith('M'):
        Table2 = pd.read_csv(file, sep=',')
        DF2 = pd.DataFrame(Table2)
        DFM = DFM.append(DF2, sort=False)

Initializing Dialogue... 
Please select a file.


In [78]:
# Deklaruje listę kolumn którę będą mi potrzebne
DFL['NMIOT'] = 1
cols1 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','NROJM1','LSUT1', 'NRMAM1', 'NOJMIO']
DFL = DFL[cols1]

In [79]:
# Zmieniam nazwy kolumn tak by odpowiadały kolumnon z DFL
DFM['LSUT1'] = np.nan  
DFM = DFM.rename(columns={'LDUR': 'LDUR1'})
DFM = DFM.rename(columns={'LDOP': 'LDUR'})
DFM = DFM.rename(columns={'LDUR1': 'LDOP'})
DFM = DFM.rename(columns={'RASAOJ': 'LRASAOJ'})
DFM = DFM.rename(columns={'NRO': 'NROJM1'})
DFM = DFM.rename(columns={'LREJ': 'LREJ1'})
DFM = DFM.rename(columns={'LWOJ': 'LWOJ1'})
# Deklaruje listę kolumn którę będą mi potrzebne
cols2 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','LSUT1','NROJM1']
DFM = DFM[cols2]

In [80]:
#LLLLL
#NOJMIO - numer ojca miotu pierwszego
#NROJM1, MRMAM1 - rodzice LPROS1 
#LDUR - DURL - data urodzenia lochy
#LDOP - DURM - data urodzenia miotu
#MMMMM
#NRO - numer ojca miotu kolejnego

In [81]:
# Rozdzielam sutki prawe od lewych
DFL['PSUT'] = DFL['LSUT1'].astype(str).str[1]
DFL['LSUT1'] = DFL['LSUT1'].astype(str).str[0]

In [82]:
# Convertuje liczby sutków do inta
convert_dict = {'LSUT1': int,
                'PSUT': int
               }
  
DFL = DFL.astype(convert_dict)

In [83]:
# Liczę sumę sutków prawych i lewych
DFL['ISUT'] = DFL['PSUT'] + DFL['LSUT1']

In [84]:
# Wprowadzamy przedział czasowy z jakiego będą pobieranie dane
date1 = input('Date range from YYYY-MM-DD') 
date2 = input('Date range to YYYY-MM-DD')

Date range from YYYY-MM-DD 2019-12-12
Date range to YYYY-MM-DD 2020-12-11


In [85]:
# Funkcja zamienia format kolumn na format daty
class ToDate():

    def change_to_date(self, a, b):
        a[b] = pd.to_datetime(a[b])


dat = ToDate()
dat.change_to_date(DFL, 'LDOP')
dat.change_to_date(DFM, 'LDOP')
dat.change_to_date(DFL, 'LDUR')
dat.change_to_date(DFM, 'LDUR')

In [86]:
# W DF zostawaimy tylko dane z wcześniej zadeklarowanego przedziału czasowego
DFL = DFL[(DFL['LDOP'] >= date1) & (DFL['LDOP'] <= date2)]
DFM = DFM[(DFM['LDOP'] >= date1) & (DFM['LDOP'] <= date2)]

In [87]:
# Wyliczamy:
# Datę pierwszego oprosienia
DFL['POP'] = DFL['LDOP'] - DFL['LDUR']
# Okres międzymiotu
DFM['OMM'] = DFM['LDOP'] - DFM['LDUR']
# Zamieniam ww wartości na dni
DFL['POP'] = DFL['POP'] / np.timedelta64(1, 'D')
DFM['OMM'] = DFM['OMM'] / np.timedelta64(1, 'D')

In [88]:
# W zmiennej DF3 łączymy poprzednie DF
DF3 = DFL.append(DFM)

In [89]:
DF3 = DF3.rename(columns={'LDUR': 'DURL'})
DF3 = DF3.rename(columns={'LDOP': 'DURM'})

In [90]:
# Wyliczamy ile prosiąt nie dożyło 21 dnia życia
DF3['STRATY'] = DF3['LIL11'] - DF3['LIL21']

In [91]:
# Wyliczamy wartość % strat
DF3['ST21'] = (DF3['STRATY'] / DF3['LIL11']) * 100

In [92]:
# Funkcja tworzy DF z określonymi już kolumnami
def createDT(name):
    temp = pd.DataFrame(columns = ['Rasa','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = name
    return temp

In [93]:
# Funkcja sortuje dane po odpowiedniej rasie
def sortedDT(DT, col, number):
    newDT = DT[DT[col] == number]
    return newDT
# Funkcja sortuje dane po rasie matki i rasie ojca
def sorted2DT(DT,number1,number2):
    newDT = DT[(DT.LRASA1 == number1) & (DT.LRASAOJ == number2)]
    return newDT

def months(DT, col, col2):
    newDT = DT[DT[col] != DT[col2]]
    return newDT


'''def birth(DFL, DFM):
    if len(DFL) > 0:
        pigs = DFL[['LIL11', 'LIL21']]
        pigs2 = DFM[['LIL11', 'LIL21']]
        pigs = pigs.append(pigs2)
        pigs['straty'] = pigs['LIL11'] - pigs['LIL21']
        pigs['ST21'] = (pigs['straty'] / pigs['LIL11']) * 100
        return pigs
    else:
        pigs = DFM[['LIL11', 'LIL21']]
        pigs['straty'] = pigs['LIL11'] - pigs['LIL21']
        pigs['ST21'] = (pigs['straty'] / pigs['LIL11']) * 100
        return pigs

    return s'''

"def birth(DFL, DFM):\n    if len(DFL) > 0:\n        pigs = DFL[['LIL11', 'LIL21']]\n        pigs2 = DFM[['LIL11', 'LIL21']]\n        pigs = pigs.append(pigs2)\n        pigs['straty'] = pigs['LIL11'] - pigs['LIL21']\n        pigs['ST21'] = (pigs['straty'] / pigs['LIL11']) * 100\n        return pigs\n    else:\n        pigs = DFM[['LIL11', 'LIL21']]\n        pigs['straty'] = pigs['LIL11'] - pigs['LIL21']\n        pigs['ST21'] = (pigs['straty'] / pigs['LIL11']) * 100\n        return pigs\n\n    return s"

In [94]:
'''hybryd1 = DF3['LRASA1']
hybryd2 = DF3['LRASAOJ']
nonHybryd = hybryd1 == hybryd2
hybryd = hybryd1!=hybryd2
DF4 = DF3[hybryd]
DF3 = DF3[nonHybryd]'''


"hybryd1 = DF3['LRASA1']\nhybryd2 = DF3['LRASAOJ']\nnonHybryd = hybryd1 == hybryd2\nhybryd = hybryd1!=hybryd2\nDF4 = DF3[hybryd]\nDF3 = DF3[nonHybryd]"

In [95]:
#'liczba ocenionych miotów ogółem' = len(DF3)
# 'liczba ocenionych miotów pierw.' = len (DF3)'NMIOT == 1 '
# liczba loch = LRPOS1.drop_duplicate()
# miedzymioty = nowa kolumna

In [96]:
# Funkcja wylicza wszystkie niezbędne wartości i przypisuje je do DF
def all_count(DF3, new):
    new['liczba loch'] = len(DF3.drop_duplicates(subset='LPROS1', keep="first").copy())
    new['liczba ocenionych miotów ogółem'] = len(DF3)
    new['liczba ocenionych miotów pierw.'] = len(DF3[DF3['NMIOT']==1])
    new['Liczba prosiąt urodzonych w miocie'] = np.around(DF3['LIL11'].mean(), decimals=2)
    new['Liczba prosiąt w 21 dniu życia'] = np.around(DF3['LIL21'].mean(), decimals=2)
    new['Straty prosiąt do 21 dnia życia'] = np.around(DF3['ST21'].mean(), decimals=2)
    new['Wiek w dniu pierwszego oprosienia(dni)'] = np.around(DF3['POP'].mean())
    new['Okres miedzymiotu(dni)'] = np.around(DF3['OMM'].mean())
    new['Liczba sutków lochy'] = np.around(DF3['ISUT'].mean(), decimals=2)

    return new

In [97]:
HAMP = createDT('Hampshire')
hamp = sortedDT(DF3,'LRASA1',60).copy()
all_count(hamp,HAMP)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,Hampshire,13,26,5,12.42,11.04,11.15,14.2,445.0,182.0


In [98]:
WBP = createDT('Wbp')
wbp = sortedDT(DF3,'LRASA1',10).copy()
all_count(wbp,WBP)

PBZ = createDT('Pbz')
pbz = sortedDT(DF3,'LRASA1',20).copy()
all_count(pbz,PBZ)

HAMP = createDT('Hampshire')
hamp = sortedDT(DF3,'LRASA1',60).copy()
all_count(hamp,HAMP)

DUR = createDT('Duroc')
dur = sortedDT(DF3,'LRASA1',70).copy()
all_count(dur,DUR)

PIET = createDT('Pietrain')
piet = sortedDT(DF3,'LRASA1',80).copy()
all_count(piet,PIET)

L99 = createDT('Linia 990')
l99 = sortedDT(DF3,'LRASA1',90).copy()
all_count(l99,L99)

ZB = createDT('Złotnicka Biała')
zb = sortedDT(DF3,'LRASA1',30).copy()
all_count(zb,ZB)

ZPS = createDT('Złotnicka Pstra')
zps = sortedDT(DF3,'LRASA1',50).copy()
all_count(zps,ZPS)

PULA = createDT('Puławska')
pula = sortedDT(DF3,'LRASA1',40).copy()
all_count(pula,PULA)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,Puławska,2110,3316,590,10.29,9.14,10.64,14.34,378.0,202.0


In [99]:
last = pd.concat([WBP,PBZ,HAMP,DUR,PIET,L99,PULA,ZB,ZPS])
last.set_index('Rasa',inplace=True)
last

,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
Rasa,,,,,,,,,
Wbp,1944,3315,736,12.48,11.65,6.26,14.99,365.0,167.0
Pbz,3104,5134,1287,12.65,11.72,6.82,15.29,359.0,173.0
Hampshire,13,26,5,12.42,11.04,11.15,14.20,445.0,182.0
Duroc,328,556,103,11.87,10.82,8.26,13.66,371.0,180.0
Pietrain,298,522,95,11.58,10.90,5.54,14.11,383.0,181.0
Linia 990,895,1580,335,9.96,9.48,4.78,14.19,374.0,155.0
Puławska,2110,3316,590,10.29,9.14,10.64,14.34,378.0,202.0
Złotnicka Biała,1134,1627,208,8.62,8.00,6.44,14.00,384.0,235.0
Złotnicka Pstra,998,1272,167,8.24,6.98,14.56,13.24,426.0,262.0


In [103]:
# Deklaruje słoknik miast
dic_miasta = {2:'Warszawa',
             4:'Białystok',
             6:'Bydgoszcz',
             8:'Gdańsk',
             10:'Katowice',
             12:'Kielce',
             14:'Koszalin',
             16:'Kraków',
             18:'Lublin',
             20:'Łódź',
             22:'Olsztyn',
             24:'Opole',
             26:'Poznań',
             28:'Rzeszów',
             32:'Wrocław',
             34:'Zielona Góra'}
# Deklaruje listę wydzielonych ras
race_list = [wbp,pbz,hamp,dur,piet,l99,zb,zps,pula]
race_names = ['WBP','PBZ','Hampshire','Duroc','Pietrain','Linia 990','PUL','ZLB','ZLP']

In [100]:
def create_cityDT(city, race):
    temp = pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = race
    temp['Miasto'] = city
    return temp

cityDF = pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])

temp =  pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])

temp_dic = {}
z = 0
for i in race_list:
        
    for x in dic_miasta:
        tempx = sortedDT(i, 'LWOJ1', x)
        
        tempy = create_cityDT(dic_miasta[x], race_names[z])
        all_count(tempx, tempy)
        temp_dic[x]=tempy
        if z < 8:
            z+=1

In [101]:
wbp_wwa = sortedDT(wbp, 'LREJ1',2)
wbp_bia = sortedDT(wbp, 'LREJ1',4)
wbp_byd = sortedDT(wbp, 'LREJ1',6)
wbp_gda = sortedDT(wbp, 'LREJ1',8)
wbp_kta = sortedDT(wbp, 'LREJ1',10)
wbp_kie = sortedDT(wbp, 'LREJ1',12)
wbp_kos = sortedDT(wbp, 'LREJ1',14)
wbp_krk = sortedDT(wbp, 'LREJ1',16)
wbp_lbl = sortedDT(wbp, 'LREJ1',18)
wbp_ldz = sortedDT(wbp, 'LREJ1',20)
wbp_ols = sortedDT(wbp, 'LREJ1',22)
wbp_opo = sortedDT(wbp, 'LREJ1',24)
wbp_poz = sortedDT(wbp, 'LREJ1',26)
wbp_rze = sortedDT(wbp, 'LREJ1',28)
wbp_wrc = sortedDT(wbp, 'LREJ1',32)
wbp_zgo = sortedDT(wbp, 'LREJ1',34)

In [102]:
wbp_wwa2 = create_cityDT('Warszawa','wbp')
wbp_bia2 = create_cityDT('Białystok','wbp')
wbp_byd2 = create_cityDT('Bydgoszcz','wbp')
wbp_gda2 = create_cityDT('Gdańsk','wbp')
wbp_kta2 = create_cityDT('Katowice','wbp')
wbp_kie2 = create_cityDT('Kielce','wbp')
wbp_kos2 = create_cityDT('Koszalin','wbp')
wbp_krk2 = create_cityDT('Kraków','wbp')
wbp_lbl2 = create_cityDT('Lublin','wbp')
wbp_ldz2 = create_cityDT('Łódź','wbp')
wbp_ols2 = create_cityDT('Olsztyn','wbp')
wbp_opo2 = create_cityDT('Opole','wbp')
wbp_poz2 = create_cityDT('Poznań','wbp')
wbp_rze2 = create_cityDT('Rzeszów','wbp')
wbp_wrc2 = create_cityDT('Wrocław','wbp')
wbp_zgo2 = create_cityDT('Zielona Góra','wbp')

In [104]:
all_count(wbp_wwa,wbp_wwa2)
all_count(wbp_bia,wbp_bia2)
all_count(wbp_byd,wbp_byd2)
all_count(wbp_gda,wbp_gda2)
all_count(wbp_kta,wbp_kta2)
all_count(wbp_kie,wbp_kie2)
all_count(wbp_kos,wbp_kos2)
all_count(wbp_krk,wbp_krk2)
all_count(wbp_lbl,wbp_lbl2)
all_count(wbp_ldz,wbp_ldz2)
all_count(wbp_ols,wbp_ols2)
all_count(wbp_opo,wbp_opo2)
all_count(wbp_poz,wbp_poz2)
all_count(wbp_rze,wbp_rze2)
all_count(wbp_wrc,wbp_wrc2)
all_count(wbp_zgo,wbp_zgo2)

,Rasa,Miasto,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,wbp,Zielona Góra,15,26,7,11.58,10.85,5.68,14.0,333.0,182.0


In [105]:
all_of_cr = pd.concat([wbp_wwa2, wbp_bia2, wbp_byd2, wbp_gda2, wbp_kta2, wbp_kie2, wbp_kos2, wbp_krk2, wbp_lbl2, wbp_ldz2, wbp_ols2, wbp_opo2, wbp_poz2, wbp_rze2, wbp_wrc2, wbp_zgo2])
all_of_cr.set_index(['Rasa','Miasto'])

liczba loch  liczba ocenionych miotów ogółem  \
Rasa Miasto                                                       
wbp  Warszawa               30                               46   
     Białystok              53                              104   
     Bydgoszcz             656                             1178   
     Gdańsk                 79                              113   
     Katowice                0                                0   
     Kielce                  0                                0   
     Koszalin                0                                0   
     Kraków                  0                                0   
     Lublin                 79                              129   
     Łódź                   66                               97   
     Olsztyn                58                              106   
     Opole                   0                                0   
     Poznań                400                              679   
     Rzeszów                80                              145   
     Wrocław                35                               60   
     Zielona Góra           15                               26   

                   liczba ocenionych miotów pierw.  \
Rasa Miasto                                          
wbp  Warszawa                                   14   
     Białystok                                   1   
     Bydgoszcz                                 236   
     Gdańsk                                     42   
     Katowice                                    0   
     Kielce                                      0   
     Koszalin                                    0   
     Kraków                                      0   
     Lublin                                     36   
     Łódź                                       38   
     Olsztyn                                    16   
     Opole                                       0   
     Poznań                                    126   
     Rzeszów                                    38   
     Wrocław                                    14   
     Zielona Góra                                7   

                   Liczba prosiąt urodzonych w miocie  \
Rasa Miasto                                             
wbp  Warszawa                                   13.15   
     Białystok                                  14.01   
     Bydgoszcz                                  12.46   
     Gdańsk                                     12.06   
     Katowice                                     NaN   
     Kielce                                       NaN   
     Koszalin                                     NaN   
     Kraków                                       NaN   
     Lublin                                     12.74   
     Łódź                                       12.55   
     Olsztyn                                    12.59   
     Opole                                        NaN   
     Poznań                                     12.64   
     Rzeszów                                    11.64   
     Wrocław                                    12.73   
     Zielona Góra                               11.58   

                   Liczba prosiąt w 21 dniu życia  \
Rasa Miasto                                         
wbp  Warszawa                               11.15   
     Białystok                              12.57   
     Bydgoszcz                              11.76   
     Gdańsk                                 10.90   
     Katowice                                 NaN   
     Kielce                                   NaN   
     Koszalin                                 NaN   
     Kraków                                   NaN   
     Lublin                                 11.09   
     Łódź                                   10.54   
     Olsztyn                                12.02   
     Opole                                    NaN   
     Poznań                                 11.90   
     Rzeszów     

In [37]:
pbz_wwa = sortedDT(pbz, 'LWOJ1',2)
pbz_kta = sortedDT(pbz, 'LWOJ1',10)
pbz_kta2 = createDT('pbz')
all_count(pbz_kta,pbz_kta2)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,pbz,194,303,94,12.89,11.12,12.51,15.22,355.0,169.0


In [38]:
pbz_wwa2 = createDT('pbz')
all_count(pbz_wwa, pbz_wwa2)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,pbz,37,70,14,14.47,13.83,3.96,15.79,401.0,154.0


In [39]:
wbp2 = createDT('wbp')

In [40]:
all_count(wbp_wwa, wbp2)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,wbp,148,245,68,13.78,13.02,5.1,15.51,384.0,160.0


In [41]:
wbp2

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,wbp,148,245,68,13.78,13.02,5.1,15.51,384.0,160.0


In [42]:
all_count(wbp_bia, city_wbp2)

NameError: name 'city_wbp2' is not defined

In [ ]:
def all_count(DF3, new):
    new['liczba loch'] = len(DF3.drop_duplicates(subset='LPROS1', keep="first").copy())
    new['liczba ocenionych miotów ogółem'] = len(DF3)
    new['liczba ocenionych miotów pierw.'] = len(DF3[DF3['NMIOT']==1])
    new['Liczba prosiąt urodzonych w miocie'] = np.around(DF3['LIL11'].mean(), decimals=2)
    new['Liczba prosiąt w 21 dniu życia'] = np.around(DF3['LIL21'].mean(), decimals=2)
    new['Straty prosiąt do 21 dnia życia'] = np.around(DF3['ST21'].mean(), decimals=2)
    new['Wiek w dniu pierwszego oprosienia(dni)'] = np.around(DF3['POP'].mean())
    new['Okres miedzymiotu(dni)'] = np.around(DF3['OMM'].mean())
    new['Liczba sutków lochy'] = np.around(DF3['ISUT'].mean(), decimals=2)

    return new

In [72]:
DF3.groupby(by = 'LWOJ1')

In [61]:
for i, sub in DF3.groupby('LWOJ1'):
    print("to jets i {}\t to jest sub {}".format(i,sub))

to jets i 2	 to jest sub            LPROS1  LRASA1       DURL       DURM  LRASAOJ  LIL11  LIL21  \
123    1069090818      50 2018-08-15 2020-08-09       50      9      9   
35     3067040618      30 2018-10-27 2020-01-17       20     11     11   
36     3067050318      30 2018-11-07 2020-01-02       20     11     11   
37     3067040718      30 2018-10-27 2020-01-04       30     11     11   
0     32252000519      10 2018-12-15 2020-01-27       20     15     15   
...           ...     ...        ...        ...      ...    ...    ...   
1735  32247036518      20 2020-03-14 2020-08-08       10     17     16   
1736  32247036818      20 2020-07-16 2020-12-11       20     20     18   
1737  32247037218      20 2019-11-21 2020-04-16       20     12     12   
1738  32247037218      20 2020-04-16 2020-09-11       20     14     14   
1746  32247089515      20 2019-08-09 2019-12-31       20      6      6   

      LWAGA1  M_WAGUR      LDBRAK  NMIOT  LREJ1  LWOJ1  LSEK1  LSTA1  \
123        0  

In [43]:
def city_count(DT,dt):
    for i, sub in dt.groupby('LWOJ1'):
        DT.loc[dic_miasta[i]]['Liczba loch'] = sub.drop_duplicates(subset='LPROS1', keep="first")(len)
        DT.loc[dic_miasta[i]]['liczba ocenionych miotów ogółem'] = len(sub)
        DT.loc[dic_miasta[i]]['liczba ocenionych miotów pierw.'] = len(sub[sub['NMIOT']==1])
        DT.loc[dic_miasta[i]]['Liczba prosiąt urodzonych w miocie'] = np.around(sub['LIL11'].mean(), decimals=2)
        DT.loc[dic_miasta[i]]['Liczba prosiąt w 21 dniu życia'] = np.around(sub['LIL21'].mean(), decimals=2)
        DT.loc[dic_miasta[i]]['Straty prosiąt do 21 dnia życia'] = np.around(sub['ST21'].mean(), decimals=2)
        DT.loc[dic_miasta[i]]['Wiek w dniu pierwszego oprosienia(dni)'] = np.around(sub['POP'].mean())
        DT.loc[dic_miasta[i]]['Okres miedzymiotu(dni)'] = np.around(sub['OMM'].mean())
        DT.loc[dic_miasta[i]]['Liczba sutków lochy'] = np.around(sub['ISUT'].mean(), decimals=2)

In [44]:
def createDT2():
    temp = pd.DataFrame(columns = ['Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'])
    temp['Miasto'] = ['Warszawa','Białystok','Bydgoszcz','Gdańsk','Katowice','Kielce','Koszalin','Kraków','Lublin','Łódź'
                     'Olsztyn','Opole','Poznań','Rzeszów','Wrocław','Zielona Góra']
    temp = pd.DataFrame(temp).set_index('Miasto')
    return temp

In [55]:
cityDF = createDT2()

In [58]:
pd.set_option('display.max_columns', None)

In [60]:
cityDF = city_count(cityDF, DF3)

TypeError: 'DataFrame' object is not callable

In [45]:
city_wbp2 = createDT2()

In [48]:
city_wbp

NameError: name 'city_wbp' is not defined

In [40]:
wbp.to_csv('Results.csv', header=True, index=False, sep=';', mode='w')